First we need to load the sequence file

In [1]:
from Bio import SeqIO

inputFile="./Data/s288c.fasta"
fasta_records = SeqIO.parse(open(inputFile),'fasta')
seqList=[]
for record in fasta_records:
    seqList.append(str(record.seq))
    

    

Then we will need to convert each sequence to codons, each codon is treated as a "word" and the sequence is the "sentence"

In [2]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './CUB_Code')
import CodonLibraries as CL

sentenceList=[]
for seq in seqList:
    codonList = CL.loadSequence(seq)
    sentence=""
    for codon in codonList:
        sentence+=codon+" "
    sentenceList.append(sentence)
#print(sentenceList[:2])

Now let's try to train a tokenlizer on our data

In [3]:
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers

tokenizer = Tokenizer(models.WordLevel())
tokenizer.normalizer = normalizers.NFKC()
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
tokenizer.decoder = decoders.ByteLevel()

trainer = trainers.WordLevelTrainer(
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
    special_tokens=['<s>', '<pad>', '<unk>', '<mask>'],
)

tokenizer.train_from_iterator(sentenceList, trainer=trainer)
import os



tokenizer.save('yeast_token')

Now the tokens are built and saved, lets try to use it

In [4]:
from transformers import BertTokenizer


# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = BertTokenizer.from_pretrained('./yeast_token')
tokenizer(sentenceList[0])


/home/lu/.local/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1661: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': [105, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [9]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import AutoTokenizer, AutoModel


model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")



id2label = {
    "0": "High",
    "1": "Mid",
    "2": "Low",
}

text ='ATGAACAGTGCTACCAGTGAGACAACAACCAATACTGGAGCTGCTGAGACAACTACCAGTACTGGAGCTGCTGAGACGAA\
AACAGTAGTCACCTCTTCAATTTCAAGATTCAATCATGCTGAAACACAGACGGCTTCCGCGACCGATGTGATTGGTCACA\
GCAGTAGTGTTGTTTCTGTATCCGAAACTGGCAACACCAAGAGTCTAATAACTTCCGGGTTAAGTACTATGTCGCAACAG\
CCTCGTAGCACACCAGCAAGTAGCATAATAGGATCTAGTACTGCCTCTTTAGAAATCTCAACCTACGTTGGTATTGCCAA\
TGGTCTGTTGACCAATAATGGCATAAGTGTTTTTATTTCCACCGTATTGCTGGCAATCGTATGGTAA'

input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)

outputs = model(input_ids)
last_hidden_states = outputs[0]

for m in last_hidden_states:
  for index, n in enumerate(m):
    if(index > 0 and index <= len(text.split(" "))):
      print(text.split(" ")[index-1] + ": " + id2label[str(torch.argmax(n).item())])
      


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyError: '112'